In [1]:
import pandas as pd
import numpy as np
#testserdar

In [2]:
wsp_contract = pd.read_excel("../../Datasets/study-1-data.xlsx")



In [3]:
wsp_contract['Sales'] = wsp_contract[['Demand', 'Stock']].min(axis=1)

In [4]:
wsp_contract['Leftovers'] = wsp_contract['Stock'] - wsp_contract['Sales']

In [5]:
wsp_contract['Unmet_Demand'] = wsp_contract['Demand'] - wsp_contract['Sales']

In [6]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Unnamed: 5,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,NaN,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,NaN,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,NaN,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,NaN,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,NaN,Retailer 1,60,60,0,75


In [7]:
wsp_contract.drop(columns=["Unnamed: 5"], inplace=True)

In [8]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75


In [9]:
wsp_contract['Realized_Mfg_Profit'] = (wsp_contract['Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Stock']) 

In [10]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300


In [11]:
wsp_contract['Total_Revenue'] = wsp_contract['Stock'] * wsp_contract['Wholesale p.']

In [12]:
wsp_contract['Realized_Mfg_Profit_Share'] = wsp_contract.apply(lambda row: row['Realized_Mfg_Profit'] / row['Total_Revenue'] if row['Total_Revenue'] != 0 else 0, axis=1)

In [13]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000


In [14]:

wsp_contract['Optimal_Stock'] = 100*((12-wsp_contract['Wholesale p.'])/12) + 50
wsp_contract['Optimal_Stock'] = wsp_contract['Optimal_Stock'].astype(int)

In [15]:
wsp_contract['Under/Overstock_wrt_Optimal'] = wsp_contract['Stock'] - wsp_contract['Optimal_Stock']

In [16]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,83,-23
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,91,-11
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,91,-11
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,75,-24
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,83,-23


In [17]:
potential_demand = np.arange(50, 151)
expected_sales = []

for index, row in wsp_contract.iterrows():
    cumulative_expected_sales = 0
    
    for demand_realization in potential_demand:
        sales = min(demand_realization, row['Stock'])

        cumulative_expected_sales += sales
        
        if sales == row['Stock']:
            cumulative_expected_sales += (150 - demand_realization) * row['Stock']
            break
    
    # Calculate the average expected sales for this row and append it to the list
    expected_sales.append(cumulative_expected_sales / len(potential_demand))

wsp_contract['Expected Sales'] = np.array(expected_sales)

In [18]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,83,-23,59.455446
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,91,-11,75.396040
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,91,-11,75.396040
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,75,-24,50.990099
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,83,-23,59.455446
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,300,480,0.625000,83,-23,59.455446
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,0,0,0.000000,58,-58,0.000000
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,306,459,0.666667,75,-24,50.990099
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,0,0,0.000000,50,-50,0.000000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,300,480,0.625000,83,-23,59.455446


In [19]:
wsp_contract["Expected_Leftovers"] = wsp_contract["Optimal_Stock"] - wsp_contract["Expected Sales"]

In [20]:
wsp_contract['Expected Retailer Profit'] = wsp_contract['Expected Sales'] * wsp_contract['Wholesale p.'] - (3 * wsp_contract['Expected Sales']) #nasıl yapacaz kaça sattığını bilmiyoz

In [21]:
wsp_contract['Expected Mfg Profit'] = (wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Optimal_Stock'])

In [22]:
wsp_contract.head()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,Realized_Mfg_Profit,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,300,480,0.625000,83,-23,59.455446,23.544554,297.277228,415
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,320,560,0.571429,91,-11,75.396040,15.603960,301.584158,364
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,320,560,0.571429,91,-11,75.396040,15.603960,301.584158,364
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,306,459,0.666667,75,-24,50.990099,24.009901,305.940594,450
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,300,480,0.625000,83,-23,59.455446,23.544554,297.277228,415


In [23]:
wsp_contract['Total Expected Revenue'] = wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']

wsp_contract['Expected Mfg. Profit Share'] = wsp_contract.apply(lambda row: row['Expected Mfg Profit'] / row['Total Expected Revenue'] if row['Total Expected Revenue'] != 0 else 0, axis=1)

In [24]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Total_Revenue,Realized_Mfg_Profit_Share,Optimal_Stock,Under/Overstock_wrt_Optimal,Expected Sales,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Total Expected Revenue,Expected Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,560,0.571429,91,-11,75.396040,15.603960,301.584158,364,637,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,560,0.571429,91,-11,75.396040,15.603960,301.584158,364,637,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,459,0.666667,75,-24,50.990099,24.009901,305.940594,450,675,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,...,0,0.000000,58,-58,0.000000,58.000000,0.000000,464,638,0.727273
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,...,459,0.666667,75,-24,50.990099,24.009901,305.940594,450,675,0.666667
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,...,0,0.000000,50,-50,0.000000,50.000000,0.000000,450,600,0.750000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,...,480,0.625000,83,-23,59.455446,23.544554,297.277228,415,664,0.625000


In [25]:
#wsp_contract.to_csv("../../Datasets/study-1-data-wsp.csv", index=False)
wsp_contract['Predicted Sales'] = wsp_contract[['Demand', 'Optimal_Stock']].min(axis=1).astype(int)

In [26]:
wsp_contract['Predicted Leftovers'] = wsp_contract['Optimal_Stock'] - wsp_contract['Predicted Sales']

In [27]:
wsp_contract['Predicted Retailer Profit'] = wsp_contract['Predicted Sales'] * wsp_contract['Wholesale p.'] - (3 * wsp_contract['Predicted Sales'])
wsp_contract['Predicted Mfg Profit'] = (wsp_contract['Optimal_Stock'] * wsp_contract['Wholesale p.']) - (3 * wsp_contract['Optimal_Stock'])

In [28]:
wsp_contract['Realized Retailer Profit'] = wsp_contract['Sales'] * 12 - (wsp_contract['Stock'] * wsp_contract['Wholesale p.'])

In [29]:
wsp_contract['Predicted Mfg. Profit Share'] = wsp_contract.apply(
    lambda row: row['Predicted Mfg Profit'] / (row['Optimal_Stock'] * row['Wholesale p.']) if (row['Optimal_Stock'] * row['Wholesale p.']) != 0 else 0,
    axis=1
)

In [30]:
wsp_contract['']

KeyError: ''

In [ ]:
wsp_contract.head(20)

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Total Expected Revenue,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Predicted Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,15.603960,301.584158,364,637,0.571429,91,0,364,364,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,15.603960,301.584158,364,637,0.571429,57,34,228,364,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,24.009901,305.940594,450,675,0.666667,75,0,450,450,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,...,58.000000,0.000000,464,638,0.727273,58,0,464,464,0.727273
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,...,24.009901,305.940594,450,675,0.666667,66,9,396,450,0.666667
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,...,50.000000,0.000000,450,600,0.750000,50,0,450,450,0.750000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,...,23.544554,297.277228,415,664,0.625000,73,10,365,415,0.625000


In [ ]:
#wsp_contract.to_csv("../../Datasets/study-1-data-wsp.csv", index=False)

In [ ]:
df1 = wsp_contract[0:40]

In [ ]:
df1.tail()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Total Expected Revenue,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Predicted Mfg. Profit Share
35,1,36,Manufacturer 1,136,9,Retailer 1,0,0,0,136,...,75.000000,0.000000,450,675,0.666667,75,0,450,450,0.666667
36,1,37,Manufacturer 1,116,8,Retailer 1,60,60,0,56,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
37,1,38,Manufacturer 1,105,8,Retailer 1,60,60,0,45,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
38,1,39,Manufacturer 1,80,8,Retailer 1,60,60,0,20,...,23.544554,297.277228,415,664,0.625000,80,3,400,415,0.625000
39,1,40,Manufacturer 1,75,8,Retailer 1,55,55,0,20,...,28.148515,274.257426,415,664,0.625000,75,8,375,415,0.625000


In [ ]:
df2 = wsp_contract[wsp_contract['Player ID'] == 'Manufacturer 1']
df2.tail()

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Total Expected Revenue,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Predicted Mfg. Profit Share
35,1,36,Manufacturer 1,136,9,Retailer 1,0,0,0,136,...,75.000000,0.000000,450,675,0.666667,75,0,450,450,0.666667
36,1,37,Manufacturer 1,116,8,Retailer 1,60,60,0,56,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
37,1,38,Manufacturer 1,105,8,Retailer 1,60,60,0,45,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
38,1,39,Manufacturer 1,80,8,Retailer 1,60,60,0,20,...,23.544554,297.277228,415,664,0.625000,80,3,400,415,0.625000
39,1,40,Manufacturer 1,75,8,Retailer 1,55,55,0,20,...,28.148515,274.257426,415,664,0.625000,75,8,375,415,0.625000


In [ ]:
dict = {}
temp = 'Manufacturer '
for i in range(1,23):
    j = str(i)
    temp += j
    dict[temp] = wsp_contract[wsp_contract['Player ID'] == temp]
    if len(j) == 1:
        temp = temp[0: -1]
    else:
        temp = temp[0: -2]

In [ ]:
len(dict['Manufacturer 20'])

40

In [ ]:
dict['Manufacturer 1']

,Experiment,Period,Player ID,Demand,Wholesale p.,Player ID.1,Stock,Sales,Leftovers,Unmet_Demand,...,Expected_Leftovers,Expected Retailer Profit,Expected Mfg Profit,Total Expected Revenue,Expected Mfg. Profit Share,Predicted Sales,Predicted Leftovers,Predicted Retailer Profit,Predicted Mfg Profit,Predicted Mfg. Profit Share
0,1,1,Manufacturer 1,122,8,Retailer 1,60,60,0,62,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
1,1,2,Manufacturer 1,144,7,Retailer 1,80,80,0,64,...,15.603960,301.584158,364,637,0.571429,91,0,364,364,0.571429
2,1,3,Manufacturer 1,57,7,Retailer 1,80,57,23,0,...,15.603960,301.584158,364,637,0.571429,57,34,228,364,0.571429
3,1,4,Manufacturer 1,149,9,Retailer 1,51,51,0,98,...,24.009901,305.940594,450,675,0.666667,75,0,450,450,0.666667
4,1,5,Manufacturer 1,135,8,Retailer 1,60,60,0,75,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
5,1,6,Manufacturer 1,94,8,Retailer 1,60,60,0,34,...,23.544554,297.277228,415,664,0.625000,83,0,415,415,0.625000
6,1,7,Manufacturer 1,143,11,Retailer 1,0,0,0,143,...,58.000000,0.000000,464,638,0.727273,58,0,464,464,0.727273
7,1,8,Manufacturer 1,66,9,Retailer 1,51,51,0,15,...,24.009901,305.940594,450,675,0.666667,66,9,396,450,0.666667
8,1,9,Manufacturer 1,123,12,Retailer 1,0,0,0,123,...,50.000000,0.000000,450,600,0.750000,50,0,450,450,0.750000
9,1,10,Manufacturer 1,73,8,Retailer 1,60,60,0,13,...,23.544554,297.277228,415,664,0.625000,73,10,365,415,0.625000


In [31]:
grouped_data_retailer = wsp_contract.groupby('Player ID')


In [32]:
for group_key, group_data in grouped_data_retailer:
    group_data['Retailer Profit Mean Of All 40 Periods'] = group_data['Realized Retailer Profit'].mean()
    group_data['Manufacturer Profit Mean Of All 40 Periods'] = group_data['Realized_Mfg_Profit'].mean()


In [33]:
group_keys = list(grouped_data_retailer.groups.keys())  # Convert to list
group_key_15 = group_keys[0]  # Assuming zero-based indexing
group_15 = grouped_data_retailer.get_group(group_key_15)


In [34]:
group_15['Realized Retailer Profit']

0     240
1     400
2     124
3     153
4     240
5     240
6       0
7     153
8       0
9     240
10    153
11    153
12    184
13    153
14    240
15      0
16    690
17    534
18    400
19    690
20    400
21    492
22    400
23    400
24     52
25    375
26    240
27    153
28    153
29    153
30    240
31    153
32    153
33    153
34      0
35      0
36    240
37    240
38    240
39    220
Name: Realized Retailer Profit, dtype: int64

In [35]:
wsp_contract.iloc[360]

Experiment                                   3
Period                                       1
Player ID                      Manufacturer 10
Demand                                     122
Wholesale p.                                 9
Player ID.1                        Retailer 10
Stock                                       90
Sales                                       90
Leftovers                                    0
Unmet_Demand                                32
Realized_Mfg_Profit                        540
Total_Revenue                              810
Realized_Mfg_Profit_Share             0.666667
Optimal_Stock                               75
Under/Overstock_wrt_Optimal                 15
Expected Sales                       81.881188
Expected_Leftovers                   -6.881188
Expected Retailer Profit            491.287129
Expected Mfg Profit                        450
Total Expected Revenue                     675
Expected Mfg. Profit Share            0.666667
Predicted Sal

In [36]:
#Null Hypothesis (H0): The mean order quantity made by retailers is not significantly different from the optimal stock level.

#Alternative Hypothesis (H1): The mean order quantity made by retailers is significantly different from the optimal stock level.

#Reasoning: Retailers are expected to be able to adapt correctly to the changing wholesale price as they are rational agents.

wsp_contract['null'] = 0
from scipy.stats import ttest_rel
t_stat, p_value = ttest_rel(wsp_contract["Under/Overstock_wrt_Optimal"], wsp_contract['null'])
print("P value is", p_value, "which is higher than 0.05.\nMeaning that the null hypothesis is accepted.")

P value is 0.10386651604983371 which is higher than 0.05.
Meaning that the null hypothesis is accepted.


In [44]:
wsp_contract = pd.read_csv("../../Notebooks/experiment/curr_data.csv")